In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import mediapipe as mp
from collections import defaultdict, deque
import math
import time
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

@dataclass
class Detection:
    """Data class for object detections"""
    bbox: List[float]  # [x1, y1, x2, y2]
    confidence: float
    class_id: int
    class_name: str
    track_id: Optional[int] = None

@dataclass
class PersonState:
    """State tracking for each person"""
    track_id: int
    held_objects: Dict[int, int] = None  # object_id -> frames_held
    pose_landmarks: Optional[List] = None
    bbox: List[float] = None
    interaction_history: List = None
    theft_flags: int = 0
    last_seen: int = 0
    
    def _post_init_(self):
        if self.held_objects is None:
            self.held_objects = {}
        if self.interaction_history is None:
            self.interaction_history = deque(maxlen=30)

class ImprovedTheftDetectionSystem:
    """
    Improved Theft Detection System with better debugging and detection capabilities
    """
    
    def _init_(self, 
                 yolo_model_path: str = 'yolov8n.pt',
                 confidence_threshold: float = 0.3,  # Lowered for better detection
                 interaction_distance_threshold: float = 150,  # Increased threshold
                 concealment_distance_threshold: float = 100,
                 min_interaction_frames: int = 3,  # Reduced for quicker detection
                 theft_confirmation_frames: int = 10,
                 debug_mode: bool = True):
        
        self.debug_mode = debug_mode
        
        # Initialize models with error handling
        try:
            print("Loading YOLOv8 model...")
            self.yolo_model = YOLO(yolo_model_path)
            print(f"✅ YOLOv8 model loaded successfully!")
            print(f"Available classes: {list(self.yolo_model.names.values())}")
            
        except Exception as e:
            print(f"❌ Error loading YOLO model: {e}")
            print("Trying to download yolov8n.pt...")
            self.yolo_model = YOLO('yolov8n.pt')  # This will auto-download
        
        try:
            print("Loading MediaPipe Pose...")
            self.mp_pose = mp.solutions.pose
            self.pose = self.mp_pose.Pose(
                static_image_mode=False,
                model_complexity=1,
                enable_segmentation=False,
                min_detection_confidence=0.3,  # Lowered threshold
                min_tracking_confidence=0.3
            )
            print("✅ MediaPipe Pose loaded successfully!")
            
        except Exception as e:
            print(f"❌ Error loading MediaPipe: {e}")
            self.pose = None
        
        # Detection parameters
        self.confidence_threshold = confidence_threshold
        self.interaction_distance_threshold = interaction_distance_threshold
        self.concealment_distance_threshold = concealment_distance_threshold
        self.min_interaction_frames = min_interaction_frames
        self.theft_confirmation_frames = theft_confirmation_frames
        
        # Tracking variables
        self.person_states: Dict[int, PersonState] = {}
        self.object_tracks: Dict[int, Dict] = {}
        self.frame_count = 0
        self.theft_alerts = []
        self.detection_stats = {
            'total_persons': 0,
            'total_objects': 0,
            'total_interactions': 0,
            'total_thefts': 0
        }
        
        # Extended list of target items for supermarket theft detection
        self.target_classes = [
            'bottle', 'cup', 'wine glass', 'fork', 'knife', 'spoon', 'bowl',
            'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 
            'hot dog', 'pizza', 'donut', 'cake', 'cell phone', 'book', 
            'clock', 'scissors', 'teddy bear', 'hair drier', 'toothbrush',
            'laptop', 'mouse', 'remote', 'keyboard', 'sports ball',
            'frisbee', 'skateboard', 'surfboard', 'tennis racket'
        ]
        
        print(f"🎯 Target classes for theft detection: {self.target_classes}")
        print("🚀 Theft Detection System initialized successfully!")
    
    def detect_objects(self, frame: np.ndarray) -> List[Detection]:
        """Detect objects using YOLOv8 with improved error handling"""
        try:
            results = self.yolo_model(frame, verbose=False, conf=self.confidence_threshold)
            detections = []
            
            for result in results:
                boxes = result.boxes
                if boxes is not None and len(boxes) > 0:
                    for box in boxes:
                        try:
                            # Extract detection data
                            bbox = box.xyxy[0].cpu().numpy().tolist()  # [x1, y1, x2, y2]
                            confidence = float(box.conf[0].cpu().numpy())
                            class_id = int(box.cls[0].cpu().numpy())
                            class_name = self.yolo_model.names[class_id]
                            
                            detection = Detection(
                                bbox=bbox,
                                confidence=confidence,
                                class_id=class_id,
                                class_name=class_name
                            )
                            detections.append(detection)
                            
                        except Exception as e:
                            if self.debug_mode:
                                print(f"Error processing detection: {e}")
                            continue
            
            # Debug output
            if self.debug_mode and self.frame_count % 30 == 0:  # Every 30 frames
                persons = [d for d in detections if d.class_name == 'person']
                objects = [d for d in detections if d.class_name in self.target_classes]
                print(f"Frame {self.frame_count}: Detected {len(persons)} persons, {len(objects)} target objects")
                if objects:
                    obj_names = [f"{obj.class_name}({obj.confidence:.2f})" for obj in objects]
                    print(f"  Objects: {obj_names}")
            
            return detections
            
        except Exception as e:
            print(f"Error in object detection: {e}")
            return []
    
    def simple_tracking(self, detections: List[Detection]) -> List[Detection]:
        """Improved tracking with better ID assignment"""
        # Separate persons and objects
        persons = [d for d in detections if d.class_name == 'person']
        objects = [d for d in detections if d.class_name in self.target_classes]
        
        # Track persons
        tracked_persons = self._track_entities(persons, 'person')
        
        # Track objects
        tracked_objects = self._track_entities(objects, 'object')
        
        # Update detection stats
        self.detection_stats['total_persons'] = len(tracked_persons)
        self.detection_stats['total_objects'] = len(tracked_objects)
        
        return tracked_persons + tracked_objects
    
    def _track_entities(self, detections: List[Detection], entity_type: str) -> List[Detection]:
        """Enhanced tracking with better matching"""
        if entity_type == 'person':
            existing_tracks = {k: v for k, v in self.person_states.items() 
                             if self.frame_count - v.last_seen < 30}  # Clean old tracks
        else:
            existing_tracks = {k: v for k, v in self.object_tracks.items() 
                             if self.frame_count - v.get('last_seen', 0) < 20}
        
        tracked_detections = []
        used_track_ids = set()
        
        for detection in detections:
            best_match_id = None
            best_score = 0
            
            # Find best matching existing track
            for track_id, track_info in existing_tracks.items():
                if track_id in used_track_ids:
                    continue
                
                if entity_type == 'person':
                    last_bbox = track_info.bbox if track_info.bbox else [0, 0, 0, 0]
                else:
                    last_bbox = track_info.get('bbox', [0, 0, 0, 0])
                
                # Calculate IoU and distance-based score
                iou = self._calculate_iou(detection.bbox, last_bbox)
                distance = self._calculate_distance(
                    self._get_bbox_center(detection.bbox), 
                    self._get_bbox_center(last_bbox)
                )
                
                # Combined score (IoU + distance factor)
                score = iou + (1.0 / (1.0 + distance / 100.0)) * 0.3
                
                if score > best_score and iou > 0.1:  # Lower IoU threshold
                    best_score = score
                    best_match_id = track_id
            
            # Assign track ID
            if best_match_id is not None:
                detection.track_id = best_match_id
                used_track_ids.add(best_match_id)
            else:
                # Create new track
                existing_ids = list(existing_tracks.keys()) + list(self.person_states.keys()) + list(self.object_tracks.keys())
                new_id = max(existing_ids + [0]) + 1
                detection.track_id = new_id
                
                if entity_type == 'person':
                    self.person_states[new_id] = PersonState(track_id=new_id)
                else:
                    self.object_tracks[new_id] = {}
            
            # Update track information
            if entity_type == 'person':
                self.person_states[detection.track_id].bbox = detection.bbox
                self.person_states[detection.track_id].last_seen = self.frame_count
            else:
                self.object_tracks[detection.track_id]['bbox'] = detection.bbox
                self.object_tracks[detection.track_id]['last_seen'] = self.frame_count
                self.object_tracks[detection.track_id]['class_name'] = detection.class_name
                self.object_tracks[detection.track_id]['confidence'] = detection.confidence
            
            tracked_detections.append(detection)
        
        return tracked_detections
    
    def _calculate_iou(self, box1: List[float], box2: List[float]) -> float:
        """Calculate Intersection over Union (IoU) of two bounding boxes"""
        try:
            x1 = max(box1[0], box2[0])
            y1 = max(box1[1], box2[1])
            x2 = min(box1[2], box2[2])
            y2 = min(box1[3], box2[3])
            
            if x2 <= x1 or y2 <= y1:
                return 0
            
            intersection = (x2 - x1) * (y2 - y1)
            area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
            area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
            union = area1 + area2 - intersection
            
            return intersection / union if union > 0 else 0
        except:
            return 0
    
    def estimate_pose(self, frame: np.ndarray, person_bbox: List[float]) -> Optional[List]:
        """Enhanced pose estimation with error handling"""
        if self.pose is None:
            return None
            
        try:
            # Extract person region with padding
            x1, y1, x2, y2 = map(int, person_bbox)
            h, w = frame.shape[:2]
            
            # Add padding and ensure bounds
            padding = 20
            x1 = max(0, x1 - padding)
            y1 = max(0, y1 - padding)
            x2 = min(w, x2 + padding)
            y2 = min(h, y2 + padding)
            
            person_roi = frame[y1:y2, x1:x2]
            
            if person_roi.size == 0 or person_roi.shape[0] < 10 or person_roi.shape[1] < 10:
                return None
            
            # Convert BGR to RGB
            rgb_roi = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
            
            # Perform pose estimation
            results = self.pose.process(rgb_roi)
            
            if results.pose_landmarks:
                # Convert normalized coordinates to frame coordinates
                landmarks = []
                for landmark in results.pose_landmarks.landmark:
                    # Convert relative coordinates to absolute
                    abs_x = landmark.x * (x2 - x1) + x1
                    abs_y = landmark.y * (y2 - y1) + y1
                    landmarks.append([abs_x, abs_y, landmark.visibility])
                
                return landmarks
            
        except Exception as e:
            if self.debug_mode:
                print(f"Pose estimation error: {e}")
        
        return None
    
    def analyze_interactions(self, persons: List[Detection], objects: List[Detection]):
        """Enhanced interaction analysis with better theft detection logic"""
        
        if self.debug_mode and persons and objects:
            print(f"Analyzing interactions: {len(persons)} persons, {len(objects)} objects")
        
        for person in persons:
            person_state = self.person_states[person.track_id]
            
            # Update pose landmarks
            if self.pose is not None:
                pose_landmarks = self.estimate_pose(self.current_frame, person.bbox)
                person_state.pose_landmarks = pose_landmarks
            
            # Check interactions with objects
            interaction_count = 0
            for obj in objects:
                if self._check_person_object_interaction(person, obj, person_state):
                    interaction_count += 1
            
            # Check for concealment (objects that disappeared)
            concealment_detected = self._check_for_concealment(person, person_state, objects)
            
            if interaction_count > 0:
                self.detection_stats['total_interactions'] += interaction_count
                
            if concealment_detected:
                self.detection_stats['total_thefts'] += 1
    
    def _check_person_object_interaction(self, person: Detection, obj: Detection, person_state: PersonState) -> bool:
        """Enhanced interaction detection with multiple criteria"""
        
        # Calculate distance between person center and object center
        person_center = self._get_bbox_center(person.bbox)
        object_center = self._get_bbox_center(obj.bbox)
        distance = self._calculate_distance(person_center, object_center)
        
        # Check if bounding boxes overlap
        bbox_overlap = self._check_bbox_overlap(person.bbox, obj.bbox)
        
        # Check if hands are near the object (if pose is available)
        hand_near_object = False
        if person_state.pose_landmarks and len(person_state.pose_landmarks) > 16:
            # Get wrist landmarks (15: left wrist, 16: right wrist)
            left_wrist = person_state.pose_landmarks[15]
            right_wrist = person_state.pose_landmarks[16]
            
            for wrist in [left_wrist, right_wrist]:
                if wrist[2] > 0.3:  # Check visibility (lowered threshold)
                    wrist_distance = self._calculate_distance([wrist[0], wrist[1]], object_center)
                    if wrist_distance < self.interaction_distance_threshold:
                        hand_near_object = True
                        break
        
        # Determine interaction (more lenient criteria)
        is_interacting = (distance < self.interaction_distance_threshold * 1.5) or bbox_overlap or hand_near_object
        
        if is_interacting:
            if obj.track_id not in person_state.held_objects:
                person_state.held_objects[obj.track_id] = 0
                if self.debug_mode:
                    print(f"👋 Person {person.track_id} started interacting with {obj.class_name} {obj.track_id}")
            
            person_state.held_objects[obj.track_id] += 1
            
            # Log interaction
            person_state.interaction_history.append({
                'frame': self.frame_count,
                'object_id': obj.track_id,
                'object_class': obj.class_name,
                'distance': distance,
                'hand_near': hand_near_object,
                'bbox_overlap': bbox_overlap
            })
            
            return True
        
        return False
    
    def _check_bbox_overlap(self, bbox1: List[float], bbox2: List[float]) -> bool:
        """Check if two bounding boxes overlap"""
        return not (bbox1[2] < bbox2[0] or bbox2[2] < bbox1[0] or 
                   bbox1[3] < bbox2[1] or bbox2[3] < bbox1[1])
    
    def _check_for_concealment(self, person: Detection, person_state: PersonState, current_objects: List[Detection]) -> bool:
        """Enhanced concealment detection with better logic"""
        
        # Get current object IDs that are still visible
        current_object_ids = set(obj.track_id for obj in current_objects)
        concealment_detected = False
        
        # Check held objects that are no longer visible
        for obj_id in list(person_state.held_objects.keys()):
            interaction_frames = person_state.held_objects[obj_id]
            
            if obj_id not in current_object_ids:
                # Object has disappeared - check if it was concealed
                if interaction_frames >= self.min_interaction_frames:
                    # Get object info
                    obj_info = self.object_tracks.get(obj_id, {})
                    object_class = obj_info.get('class_name', 'unknown')
                    confidence = obj_info.get('confidence', 0.5)
                    
                    # Check concealment gesture
                    concealment_confidence = self._analyze_concealment_gesture(person, person_state)
                    
                    # Lower threshold for detection
                    if concealment_confidence > 0.3:  # More lenient threshold
                        person_state.theft_flags += 1
                        concealment_detected = True
                        
                        alert = {
                            'frame': self.frame_count,
                            'person_id': person.track_id,
                            'object_id': obj_id,
                            'object_class': object_class,
                            'interaction_frames': interaction_frames,
                            'concealment_confidence': concealment_confidence,
                            'detection_confidence': confidence,
                            'bbox': person.bbox.copy()
                        }
                        self.theft_alerts.append(alert)
                        
                        print(f"🚨 THEFT DETECTED! Person {person.track_id} concealed {object_class} at frame {self.frame_count}")
                        print(f"   Interaction frames: {interaction_frames}, Concealment confidence: {concealment_confidence:.2f}")
                
                # Remove the object from held_objects
                del person_state.held_objects[obj_id]
        
        return concealment_detected
    
    def _analyze_concealment_gesture(self, person: Detection, person_state: PersonState) -> float:
        """Analyze concealment gesture and return confidence score"""
        
        base_confidence = 0.5  # Base confidence if no pose data
        
        if not person_state.pose_landmarks or len(person_state.pose_landmarks) < 25:
            return base_confidence
        
        try:
            # Get relevant body landmarks
            left_wrist = person_state.pose_landmarks[15]
            right_wrist = person_state.pose_landmarks[16]
            left_hip = person_state.pose_landmarks[23]
            right_hip = person_state.pose_landmarks[24]
            
            concealment_indicators = 0
            total_checks = 0
            
            # Check if hands are near torso/hip area
            for wrist in [left_wrist, right_wrist]:
                if wrist[2] > 0.3:  # Check visibility
                    for hip in [left_hip, right_hip]:
                        if hip[2] > 0.3:
                            distance = self._calculate_distance([wrist[0], wrist[1]], [hip[0], hip[1]])
                            total_checks += 1
                            if distance < self.concealment_distance_threshold:
                                concealment_indicators += 1
            
            # Calculate confidence based on gesture analysis
            if total_checks > 0:
                gesture_confidence = concealment_indicators / total_checks
                return max(base_confidence, gesture_confidence)
            else:
                return base_confidence
                
        except Exception as e:
            if self.debug_mode:
                print(f"Concealment analysis error: {e}")
            return base_confidence
    
    def _get_bbox_center(self, bbox: List[float]) -> List[float]:
        """Get center point of bounding box"""
        return [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]
    
    def _calculate_distance(self, point1: List[float], point2: List[float]) -> float:
        """Calculate Euclidean distance between two points"""
        return math.sqrt((point1[0] - point2[0])*2 + (point1[1] - point2[1])*2)
    
    def draw_detections(self, frame: np.ndarray, persons: List[Detection], objects: List[Detection]) -> np.ndarray:
        """Enhanced visualization with better debugging info"""
        
        # Draw persons with detailed status
        for person in persons:
            person_state = self.person_states.get(person.track_id)
            color = (0, 255, 0)  # Green by default
            thickness = 2
            
            # Change color and thickness based on status
            if person_state and person_state.theft_flags > 0:
                color = (0, 0, 255)  # Red for theft
                thickness = 3
            elif person_state and person_state.held_objects:
                color = (0, 255, 255)  # Yellow for interaction
                thickness = 2
            
            # Draw bounding box
            x1, y1, x2, y2 = map(int, person.bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            
            # Draw detailed label
            label_parts = [f"Person {person.track_id}"]
            
            if person_state:
                if person_state.theft_flags > 0:
                    label_parts.append("[THEFT DETECTED!]")
                if person_state.held_objects:
                    held_items = [self.object_tracks.get(obj_id, {}).get('class_name', f'obj_{obj_id}') 
                                 for obj_id in person_state.held_objects.keys()]
                    label_parts.append(f"[Holding: {', '.join(held_items)}]")
            
            label = " ".join(label_parts)
            
            # Multi-line label if too long
            if len(label) > 50:
                lines = [label[:50], label[50:]]
            else:
                lines = [label]
            
            for i, line in enumerate(lines):
                cv2.putText(frame, line, (x1, y1-10-i*20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Draw pose landmarks if available
            if person_state and person_state.pose_landmarks:
                self._draw_pose_landmarks(frame, person_state.pose_landmarks)
        
        # Draw objects with confidence scores
        for obj in objects:
            x1, y1, x2, y2 = map(int, obj.bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue
            
            label = f"{obj.class_name} {obj.track_id} ({obj.confidence:.2f})"
            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        # Draw theft alerts with animation
        if self.theft_alerts:
            latest_alert = self.theft_alerts[-1]
            frames_since_alert = self.frame_count - latest_alert['frame']
            if frames_since_alert < 90:  # Show for 3 seconds at 30fps
                # Blinking effect
                if frames_since_alert % 20 < 10:
                    cv2.putText(frame, "⚠️ THEFT DETECTED! ⚠️", (50, 50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                    
                    # Show details
                    details = f"Person {latest_alert['person_id']} - {latest_alert['object_class']}"
                    cv2.putText(frame, details, (50, 90), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        
        # Enhanced status information
        info_lines = [
            f"Frame: {self.frame_count}",
            f"Persons: {len(persons)} | Objects: {len(objects)}",
            f"Interactions: {self.detection_stats['total_interactions']} | Thefts: {len(self.theft_alerts)}",
            f"Confidence Threshold: {self.confidence_threshold}",
        ]
        
        for i, line in enumerate(info_lines):
            y_pos = frame.shape[0] - 80 + i * 20
            cv2.putText(frame, line, (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def _draw_pose_landmarks(self, frame: np.ndarray, landmarks: List):
        """Draw key pose landmarks"""
        # Draw key points (hands and hips)
        key_points = [15, 16, 23, 24]  # Left wrist, Right wrist, Left hip, Right hip
        colors = [(0, 255, 0), (0, 255, 0), (255, 0, 255), (255, 0, 255)]
        
        for idx, color in zip(key_points, colors):
            if idx < len(landmarks) and landmarks[idx][2] > 0.3:  # Check visibility
                x, y = int(landmarks[idx][0]), int(landmarks[idx][1])
                cv2.circle(frame, (x, y), 6, color, -1)
    
    def process_video(self, video_path: str, output_path: str = None, display: bool = True, skip_frames: int = 0):
        """Process video with enhanced error handling and performance"""
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Error: Could not open video {video_path}")
            return
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"📹 Processing video: {width}x{height} @ {fps} FPS, Total frames: {total_frames}")
        
        # Setup video writer if output path provided
        out = None
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
            print(f"💾 Output will be saved to: {output_path}")
        
        self.frame_count = 0
        start_time = time.time()
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Skip frames for performance if needed
                if skip_frames > 0 and self.frame_count % (skip_frames + 1) != 0:
                    self.frame_count += 1
                    continue
                
                self.current_frame = frame
                
                # Detect objects
                detections = self.detect_objects(frame)
                
                # Track objects
                tracked_detections = self.simple_tracking(detections)
                
                # Separate persons and objects
                persons = [d for d in tracked_detections if d.class_name == 'person']
                objects = [d for d in tracked_detections if d.class_name in self.target_classes]
                self.current_objects = objects
                
                # Analyze interactions and detect theft
                if persons and objects:  # Only analyze if both persons and objects are present
                    self.analyze_interactions(persons, objects)
                
                # Draw results
                output_frame = self.draw_detections(frame, persons, objects)
                
                # Write frame if output specified
                if out:
                    out.write(output_frame)
                
                # Display frame
                if display:
                    cv2.imshow('Enhanced Theft Detection System', output_frame)
                    
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('q'):
                        print("👋 Stopping video processing...")
                        break
                    elif key == ord('s'):
                        # Save screenshot
                        screenshot_name = f'theft_detection_frame_{self.frame_count}.jpg'
                        cv2.imwrite(screenshot_name, output_frame)
                        print(f"📸 Screenshot saved: {screenshot_name}")
                    elif key == ord('d'):
                        # Toggle debug mode
                        self.debug_mode = not self.debug_mode
                        print(f"🔧 Debug mode: {'ON' if self.debug_mode else 'OFF'}")
                
                self.frame_count += 1
                
                # Print progress
                if self.frame_count % 100 == 0:
                    elapsed = time.time() - start_time
                    fps_current = self.frame_count / elapsed
                    progress = (self.frame_count / total_frames) * 100
                    print(f"📊 Progress: {progress:.1f}% | FPS: {fps_current:.1f} | Thefts: {len(self.theft_alerts)}")
        
        except Exception as e:
            print(f"❌ Error during video processing: {e}")
        
        finally:
            # Cleanup
            cap.release()
            if out:
                out.release()
            if display:
                cv2.destroyAllWindows()
            
            # Print final results
            elapsed_total = time.time() - start_time
            print(f"\n{'='*50}")
            print(f"🎯 THEFT DETECTION RESULTS")
            print(f"{'='*50}")
            print(f"📹 Total frames processed: {self.frame_count}")
            print(f"⏱️  Total processing time: {elapsed_total:.2f} seconds")
            print(f"🎬 Average FPS: {self.frame_count / elapsed_total:.2f}")
            print(f"👥 Peak persons detected: {self.detection_stats['total_persons']}")
            print(f"📦 Peak objects detected: {self.detection_stats['total_objects']}")
            print(f"🤝 Total interactions: {self.detection_stats['total_interactions']}")
            print(f"🚨 Total theft alerts: {len(self.theft_alerts)}")
            
            if self.theft_alerts:
                print(f"\n📋 DETAILED THEFT EVENTS:")
                for i, alert in enumerate(self.theft_alerts, 1):
                    print(f"  {i}. Frame {alert['frame']:6d}: Person {alert['person_id']} concealed {alert['object_class']}")
                    print(f"     Confidence: {alert['concealment_confidence']:.2f} | Interaction frames: {alert['interaction_frames']}")
            else:
                print(f"✅ No theft detected in this video.")
    
    def process_webcam(self, camera_id: int = 0):
        """Process live webcam feed for real-time theft detection"""
        
        cap = cv2.VideoCapture(camera_id)
        if not cap.isOpened():
            print(f"❌ Error: Could not open camera {camera_id}")
            return
        
        # Set camera properties for better performance
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        print("🎥 Starting real-time theft detection from webcam...")
        print("Controls: 'q' to quit, 's' to save screenshot, 'd' to toggle debug")
        
        self.frame_count = 0
        start_time = time.time()
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("❌ Failed to capture frame")
                    break
                
                self.current_frame = frame
                
                # Detect and track
                detections = self.detect_objects(frame)
                tracked_detections = self.simple_tracking(detections)
                
                # Separate persons and objects
                persons = [d for d in tracked_detections if d.class_name == 'person']
                objects = [d for d in tracked_detections if d.class_name in self.target_classes]
                self.current_objects = objects
                
                # Analyze interactions
                if persons and objects:
                    self.analyze_interactions(persons, objects)
                
                # Draw and display
                output_frame = self.draw_detections(frame, persons, objects)
                cv2.imshow('Real-time Theft Detection', output_frame)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    screenshot_name = f'realtime_theft_detection_{int(time.time())}.jpg'
                    cv2.imwrite(screenshot_name, output_frame)
                    print(f"📸 Screenshot saved: {screenshot_name}")
                elif key == ord('d'):
                    self.debug_mode = not self.debug_mode
                    print(f"🔧 Debug mode: {'ON' if self.debug_mode else 'OFF'}")
                
                self.frame_count += 1
                
                # Print stats every 5 seconds
                if self.frame_count % 150 == 0:  # Assuming 30 FPS
                    elapsed = time.time() - start_time
                    fps_current = self.frame_count / elapsed
                    print(f"📊 Live Stats - FPS: {fps_current:.1f} | Persons: {len(persons)} | Objects: {len(objects)} | Thefts: {len(self.theft_alerts)}")
        
        except Exception as e:
            print(f"❌ Error during webcam processing: {e}")
        
        finally:
            cap.release()
            cv2.destroyAllWindows()
    
    def test_detection_on_frame(self, frame_path: str):
        """Test detection capabilities on a single frame"""
        print(f"🧪 Testing detection on frame: {frame_path}")
        
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"❌ Could not load image: {frame_path}")
            return
        
        # Detect objects
        detections = self.detect_objects(frame)
        
        print(f"📊 Detection Results:")
        print(f"  Total detections: {len(detections)}")
        
        persons = [d for d in detections if d.class_name == 'person']
        objects = [d for d in detections if d.class_name in self.target_classes]
        
        print(f"  Persons: {len(persons)}")
        print(f"  Target objects: {len(objects)}")
        
        if persons:
            print("  Person details:")
            for person in persons:
                print(f"    - Person: confidence={person.confidence:.2f}, bbox={person.bbox}")
        
        if objects:
            print("  Object details:")
            for obj in objects:
                print(f"    - {obj.class_name}: confidence={obj.confidence:.2f}, bbox={obj.bbox}")
        
        # Draw detections
        output_frame = frame.copy()
        for person in persons:
            x1, y1, x2, y2 = map(int, person.bbox)
            cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(output_frame, f"Person {person.confidence:.2f}", (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        for obj in objects:
            x1, y1, x2, y2 = map(int, obj.bbox)
            cv2.rectangle(output_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(output_frame, f"{obj.class_name} {obj.confidence:.2f}", (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        
        # Display result
        cv2.imshow('Detection Test', output_frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        # Save test result
        test_output_path = f"detection_test_{int(time.time())}.jpg"
        cv2.imwrite(test_output_path, output_frame)
        print(f"💾 Test result saved to: {test_output_path}")

# Debugging and troubleshooting functions
def diagnose_system():
    """Diagnose common issues with the theft detection system"""
    print("🔍 SYSTEM DIAGNOSIS")
    print("="*50)
    
    # Check PyTorch
    try:
        import torch
        print(f"✅ PyTorch: {torch._version_}")
        print(f"   CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"   CUDA device: {torch.cuda.get_device_name()}")
    except ImportError:
        print("❌ PyTorch not installed")
    
    # Check OpenCV
    try:
        import cv2
        print(f"✅ OpenCV: {cv2._version_}")
    except ImportError:
        print("❌ OpenCV not installed")
    
    # Check Ultralytics
    try:
        from ultralytics import YOLO
        print("✅ Ultralytics YOLO available")
    except ImportError:
        print("❌ Ultralytics not installed")
    
    # Check MediaPipe
    try:
        import mediapipe as mp
        print("✅ MediaPipe available")
    except ImportError:
        print("❌ MediaPipe not installed")
    
    print("\n📋 INSTALLATION COMMANDS:")
    print("pip install ultralytics opencv-python mediapipe torch numpy")
    
    # Test YOLO model download
    try:
        print("\n🔄 Testing YOLO model download...")
        model = YOLO('yolov8n.pt')
        print("✅ YOLO model loaded successfully")
        print(f"   Available classes: {len(model.names)} classes")
        print(f"   Sample classes: {list(model.names.values())[:10]}")
    except Exception as e:
        print(f"❌ YOLO model test failed: {e}")

# Enhanced example usage
def main():
    """Enhanced example usage with debugging options"""
    
    print("🚀 ENHANCED THEFT DETECTION SYSTEM")
    print("="*50)
    
    # Run system diagnosis first
    diagnose_system()
    
    # Initialize the system with debugging enabled
    detector = ImprovedTheftDetectionSystem(
        confidence_threshold=0.3,  # Lower threshold for better detection
        interaction_distance_threshold=150,  # Larger interaction zone
        concealment_distance_threshold=100,
        min_interaction_frames=2,  # Faster detection
        theft_confirmation_frames=8,
        debug_mode=True  # Enable debugging
    )
    
    # Option 1: Test with a single frame first
    test_frame_path = "test_frame.jpg"  # Replace with your test image
    try:
        detector.test_detection_on_frame(test_frame_path)
    except Exception as e:
        print(f"Skipping frame test: {e}")
    
    # Option 2: Process a video file
    video_path = "supermarket_theft_video.mp4"  # Replace with your video path
    output_path = "theft_detection_output.mp4"
    
    try:
        print(f"\n🎬 Starting video processing...")
        detector.process_video(
            video_path=video_path, 
            output_path=output_path, 
            display=True,
            skip_frames=0  # Process all frames, set to 1 to skip every other frame for speed
        )
    except FileNotFoundError:
        print(f"❌ Video file not found: {video_path}")
        print("🎥 Trying webcam instead...")
        
        # Option 3: Process webcam if video file not found
        try:
            detector.process_webcam(camera_id=0)
        except Exception as e:
            print(f"❌ Webcam error: {e}")
    
    except Exception as e:
        print(f"❌ Error processing video: {e}")
        print("💡 Try reducing the confidence threshold or checking your video format")

# Additional utility function for batch processing
def process_multiple_videos(video_paths: List[str], output_dir: str = "theft_detection_outputs"):
    """Process multiple videos in batch"""
    import os
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    detector = ImprovedTheftDetectionSystem(debug_mode=True)
    
    for i, video_path in enumerate(video_paths, 1):
        print(f"\n🎬 Processing video {i}/{len(video_paths)}: {video_path}")
        
        video_name = os.path.basename(video_path).split('.')[0]
        output_path = os.path.join(output_dir, f"{video_name}_theft_detection.mp4")
        
        try:
            detector.process_video(video_path, output_path, display=False)
        except Exception as e:
            print(f"❌ Error processing {video_path}: {e}")
            continue

if _name_ == "_main_":
    main()